<a href="https://colab.research.google.com/github/galavidar/lab-project/blob/main/Matching_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## CV Job Matching using Doc2Vec

### Coding
#### 1. Set up

In [1]:
# Install all dependencies
!pip install gensim
!pip install nltk
!pip install pandas
!pip install numpy
!pip install requests
!pip install playwright
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 9.7 MB/s eta 0:00:00
161.3 MiB [] 0% 0.0s161.3 MiB [] 0% 37.4s161.3 MiB [] 0% 19.5s161.3 MiB [] 0% 13.4s161.3 MiB [] 0% 8.2s161.3 MiB [] 1% 7.1s161.3 MiB [] 1% 6.7s161.3 MiB [] 1% 6.0s161.3 MiB [] 2% 5.7s161.3 MiB [] 2% 5.3s161.3 MiB [] 3% 5.3s161.3 MiB [] 3% 4.9s161.3 MiB [] 4% 4.5s161.3 MiB [] 4% 4.3s161.3 MiB [] 5% 4.3s161.3 MiB [] 5% 4.1s161.3 MiB [] 6% 4.0s161.3 MiB [] 6% 3.9s161.3 MiB [] 6% 4.4s161.3 MiB [] 7% 4.2s161.3 MiB [] 8% 4.0s161.3 MiB [] 8% 3.9s161.3 MiB [] 9% 3.9s161.3 MiB [] 9% 3.7s161.3 MiB [] 10% 3.6s161.3 MiB [] 11% 3.5s161.3 MiB [] 12% 3.4s161.3 MiB [] 13% 3.2s161.3 MiB [] 14% 3.0s161.3 MiB [] 14% 3.2s161.3 MiB [] 15% 3.0s161.3 MiB [] 16% 2.9s161.3 MiB [] 17% 2.8s161.3 MiB [] 18% 2.7s161.3 MiB [] 19% 2.6s161.3 MiB [] 21% 2.4s161.3 MiB [] 22% 2.3s161.3 MiB [] 23% 2.3s161.3 MiB [] 24% 2.2s161.3 MiB [] 26% 2.1s161.3 MiB [] 27% 2.0s161.3 MiB [] 28% 1.9s161.3 MiB [] 29% 1.9s161.3 MiB [] 29% 2.0s161.3 MiB [] 29% 2.

In [2]:
# Import libraries
from gensim.models.doc2vec import Doc2Vec, TaggedDocument,Word2Vec
from nltk.tokenize import word_tokenize
from numpy.linalg import norm
from termcolor import colored
import pandas as pd
import numpy as np
import requests
import re
import plotly.graph_objects as go
import nltk
nltk.download('all', download_dir='/usr/local/lib/nltk_data')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /usr/local/lib/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package avera

True

In [3]:
# Load data
jobs = pd.read_csv('/content/linkedin_tech_82k_git.csv')
profiles=pd.read_csv('/content/Lab_project.csv')

Keep only some columns to train the model

In [4]:
jobs =jobs[['Employment type', 'Industries', 'Seniority level',
       'company', 'description', 'education',
       'location', 'months_experience', 'post_url', 'salary', 'title']]

profiles=profiles[['about', 'certifications',
       'current_company',
       'education', 'experience','languages',
       'position', 'url', 'volunteer_experience', 'сourses']]


In [5]:
# Create a new column called 'data' and merge the values of the other columns into it
jobs['data'] = jobs[['Employment type', 'Industries', 'Seniority level',
       'company', 'description', 'education',
       'location', 'months_experience', 'post_url', 'salary', 'title']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
# Drop the individual columns if you no longer need them
jobs.drop(['Employment type', 'Industries', 'Seniority level',
       'company', 'description', 'education',
       'location', 'months_experience', 'post_url', 'salary', 'title'], axis=1, inplace=True)
# Preview the updated dataframe
print(jobs.head())

                                                data
0  Full-time Broadcast Media Mid-Senior level Cyb...
1  Full-time Hospital & Health Care, Medical Devi...
2  Full-time Computer Hardware, Computer Software...
3  Full-time Computer Hardware, Computer Software...
4  Full-time Computer Hardware, Computer Software...


In [6]:
# Create a new column called 'data' and merge the values of the other columns into it
profiles['data'] = profiles[['about', 'certifications',
       'current_company',
       'education', 'experience','languages',
       'position', 'url', 'volunteer_experience', 'сourses']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
# Drop the individual columns if you no longer need them
profiles.drop(['about', 'certifications',
       'current_company',
       'education', 'experience','languages',
       'position', 'url', 'volunteer_experience', 'сourses'], axis=1, inplace=True)
# Preview the updated dataframe
print(profiles.head())

                                                data
0  [] {"company_id":null,"industry":"Michigan Con...
1  [] {"company_id":"bcbsks","industry":"Blue Cro...
2  [] {"company_id":null,"industry":"Holm Corruga...
3  [{"meta":"Issued Sep 2021 See credential","sub...
4  [] {"company_id":"new-york-college-of-traditio...


#### 3. Tokenize data
We tokenize the words in the 'data' column and tag them with unique identifiers using the TaggedDocument class.

In [7]:
jobs_data = list(jobs['data'])
user_data=(list(profiles['data']))



In [8]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Load NLTK stopwords
stop_words = set(stopwords.words('english'))

def preprocess_data(text):
    """
    Tokenize and clean text by removing stopwords, punctuation, and non-alphabetic tokens.
    """
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    filtered_tokens = [
        word for word in tokens
        if word.isalpha() and word not in stop_words  # Keep only alphabetic words and non-stopwords
    ]
    return filtered_tokens


In [9]:
# Preprocess and tag profiles
tagged_profiles = [
    TaggedDocument(words=preprocess_data(profile), tags=[f"profile_{i}"])
    for i, profile in enumerate(user_data)
]

# Preprocess and tag jobs
tagged_jobs = [
    TaggedDocument(words=preprocess_data(job), tags=[f"job_{i}"])
    for i, job in enumerate(jobs_data)
]

# Combine both lists
tagged_data = tagged_profiles + tagged_jobs


KeyboardInterrupt: 

In [10]:
from gensim.models import Word2Vec

# Preprocess and tokenize profiles and jobs
tokenized_profiles = [preprocess_data(profile) for profile in user_data]
tokenized_jobs = [preprocess_data(job) for job in jobs_data]

# Combine all tokenized data
tokenized_data = tokenized_profiles + tokenized_jobs

# Train Word2Vec model
model = Word2Vec(sentences=tokenized_data, vector_size=100, min_count=2, epochs=10, workers=4)

# Get the vocabulary keys
keys = model.wv.key_to_index.keys()

# Print the length of the vocabulary keys
print(len(keys))


20041


In [44]:
model = Word2Vec(vector_size=100, min_count=2, epochs=10, workers=4)

In [45]:
# Vocabulary building
model.build_vocab(tagged_data)
# Get the vocabulary keys
keys = model.wv.key_to_index.keys()
# Print the length of the vocabulary keys
print(len(keys))

TypeError: unhashable type: 'list'

#### 5. Train and save the model
Train the model on tagged data.

In [12]:
# Train the model
for epoch in range(model.epochs):
    print(f"Training epoch {epoch+1}/{model.epochs}")
# Train for all desired epochs in a single call
    model.train(
    tokenized_data,
    total_examples=model.corpus_count,
    epochs=10,  # Set the total number of training epochs
    start_alpha=0.025,
    end_alpha=0.0001
)

model.save('cv_job_maching.model')
print("Model saved")

Training epoch 1/10


Training epoch 2/10


Training epoch 3/10


Training epoch 4/10


Training epoch 5/10


Training epoch 6/10


Training epoch 7/10


Training epoch 8/10


Training epoch 9/10


Training epoch 10/10
Model saved


#### 6. Inputs of CV and JD

In [34]:
profile_experiance= """Company Name: Next Career
Duration at company: 2 yrs 1 mo
Job Name: Headhunter (via Next Career)
Company: Flex
Job Type: Contract
Job Duration: Apr 2024 - May 2024 · 2 mos
Job Location: Israel · Hybrid
Additional Content: Recruitment as a Service
Skills: N/A
--------------------------------------------------
Job Name: Headhunter (via Next Career)
Company: Puzzle Projects LTD
Job Type: Contract
Job Duration: Feb 2024 - Apr 2024 · 3 mos
Job Location: Israel · Remote
Additional Content: Recruitment as a service for airspace and defence company.
Skills: N/A
--------------------------------------------------
Job Name: Talent Acquisition Manager
Company: eBay & Bitfury (via Contract)
Job Type: Full-time
Job Duration: 2017 - 2019 · 2 yrs
Job Location: Israel
Additional Content: Managing the full-cycle recruiting process through an outsourcing company.Conducting phone interviews and face-to-face HR interviews.Experience in global recruitment (EMEA, APAC).Proven experience with sourcing techniques: LinkedIn recruiter corporate, amazing-hiring, X-ray search, Social media.Working closely with hiring managers and stakeholders and leading the hiring process.Hiring for Different RnD roles: DevOps, Backend development, and more.
Skills: Communication · Hiring · Recruiting · Strategic Sourcing · Sourcing
--------------------------------------------------
Job Name: Senior Talent Acquisition Specialist
Company: Israeli Navy
Job Type: Full-time
Job Duration: 2016 - 2017 · 1 yr
Job Location: Israel
Additional Content: In charge of recruitment to classified sections in the Israeli navy, units such as Mafteh.Sourcing & Hiring candidates mostly from a technological background in fields of Backend Development: (Java, Python, NodeJS, GO), FrontEnd: (HTML, Angular, JS), DevOps, Different IT positions.
Skills: N/A
--------------------------------------------------"""

profile_education="""
Education record:
  Institution: University of HaifaUniversity of Haifa
  Date: 2019 - 2021
  Description: Master's degree, Organizational Behavior StudiesMaster's degree, Organizational Behavior Studies
  Skills:
  Additional Text: Grade: 94
Education record:
  Institution: University of HaifaUniversity of Haifa
  Date: 2012 - 2014
  Description: Bachelor's degree, Human Resources Management and ServicesBachelor's degree, Human Resources Management and Services
  Skills:
  Additional Text: Grade: 95
Education record:
  Institution: University of HaifaUniversity of Haifa
  Date: 2012 - 2014
  Description: Bachelor's degree, National Security Policy StudiesBachelor's degree, National Security Policy Studies
  Skills:
  Additional Text:
 """

profile_skills="""Unique skills: ['Analytical Skills', 'Candidate Management', 'Co-sourcing', 'Communication', 'Corporate Recruiting', 'Curriculum Vitae (CV)', 'English', 'Executive Search', 'Game Theory', 'Hiring', 'Human Resources (HR)', 'Interpersonal Skills', 'Interview Preparation', 'Leadership', 'LinkedIn', 'Microsoft Excel', 'Problem Solving', 'Recruiting', 'Recruitment-to-Recruitment', 'Resume Writing', 'Sales', 'Sourcing', 'Start-up Ventures', 'Statistical Data Analysis', 'Statistics', 'Strategic Sourcing', 'Strategy', 'Vacancies', 'Venture Capital']]"""

In [35]:
from playwright.async_api import async_playwright
import asyncio

async def scrape_job_posting(job_url):
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0")
        page = await context.new_page()

        await page.goto(job_url)
        await asyncio.sleep(5)  # Wait for 5 seconds

        # Saving the page's HTML content for debugging purposes
        html = await page.content()
        with open("page_debug.html", "w", encoding="utf-8") as f:
            f.write(html)
        print("Saved page content to page_debug.html")

        # Taking a screenshot of the page for verification
        await page.screenshot(path="screenshot.png", full_page=True)
        print("Screenshot saved as screenshot.png")


        # Extract job title
        job_title = await page.inner_text('h1.top-card-layout__title')  # Adjust selector
        company = await page.inner_text('a.topcard__org-name-link')  # Adjust selector
        location = await page.inner_text('span.topcard__flavor--bullet')  # Adjust selector
        description = await page.inner_text('div.show-more-less-html__markup')  # Adjust selector

        print(f"Job Title: {job_title}")
        print(f"Company: {company}")
        print(f"Location: {location}")
        print(f"Description: {description}")

        await browser.close()
        return job_title, company, location, description

job_url = "https://www.linkedin.com/jobs/view/4119256367"
job_title, company, location, description = await scrape_job_posting(job_url)

Saved page content to page_debug.html
Screenshot saved as screenshot.png
Job Title: Recruitment Lead
Company: Logistics company (NDA) 
Location:  Tel Aviv District, Israel
Description: HR Operations (Recruitment Lead)




We're seeking an HR Operations Manager in Tel Aviv.




You will be responsible for:

• Leading the recruitment team and overseeing mass recruitment for our business units

• Managing new-hire orientation and onboarding

Analyzing and optimizing business processes, advising on local law, and collaborating with providers

• Implementing an effective onboarding system

• Working with team metrics and KPIs

• Developing a high-quality recruitment strategy and structure




You might be a fit if you:

• Have solid relevant experience in talent development or recruitment

• Have extensive knowledge of local employment law

• Are a strong team player

• Are experienced in supporting various HR projects and drafting or amending policies based on business needs

• Are able to

- **Develop a function to pre-process input text**:

In [36]:
parts = [profile_experiance, profile_education, profile_skills]
profile_combined = " ".join(parts)

parts2 = [job_title, company, location, description]
jobs_combined = " ".join(parts2)

In [40]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    # Ensure the input is a string (not tokenized yet)
    if isinstance(text, list):  # If it's already tokenized, return as-is
        return text
    # Lowercase and remove non-alphanumeric characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    # Tokenize
    words = text.split()
    # Remove stopwords
    words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    return words


In [41]:
# Apply to CV and JD
input_CV = preprocess_text(profile_combined)
input_JD = preprocess_text(jobs_combined)

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

def get_tfidf_weighted_vector(model, document, tfidf_weights):
    words = document.split()
    word_vectors = [model.wv[word] * tfidf_weights.get(word, 0) for word in words if word in model.wv]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Example TF-IDF setup
tfidf = TfidfVectorizer()
tfidf.fit([input_CV, input_JD])  # Fit on both the CV and Job Description
tfidf_weights = dict(zip(tfidf.get_feature_names_out(), tfidf.idf_))


AttributeError: 'list' object has no attribute 'lower'

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('all-MiniLM-L6-v2')  # Pretrained model for semantic similarity
v1 = model.encode(input_CV)
v2 = model.encode(input_JD)

similarity = cosine_similarity([v1], [v2])[0][0] * 100
print(f"Similarity: {round(similarity, 2)}")


#### 7. Matching
Using the trained model, we infer the document vectors for the resume and job description. Then, we calculate the cosine similarity between the two vectors to determine the match between the resume and the job description.

In [18]:
# Model evaluation
model = Doc2Vec.load('cv_job_maching.model')
v1 = model.infer_vector(input_CV.split())
v2 = model.infer_vector(input_JD.split())
similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
print(round(similarity, 2))

AttributeError: 'Word2Vec' object has no attribute 'infer_vector'

In [29]:
from numpy.linalg import norm
import numpy as np

# Load the trained Word2Vec model
model = Word2Vec.load('cv_job_maching.model')

def get_document_vector(model, document):
    """
    Compute the document vector by averaging word vectors.
    Words not in the model's vocabulary are ignored.
    """
    words = document.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if not word_vectors:  # Handle empty or unknown words
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

# Compute document vectors for the input CV and Job Description
v1 = get_document_vector(model, input_CV)
v2 = get_document_vector(model, input_JD)

# Compute cosine similarity
similarity = 100 * np.dot(v1, v2) / (norm(v1) * norm(v2))
print(round(similarity, 2))


29.16


In [24]:
print("v1 (CV vector):", v1)
print("v2 (JD vector):", v2)


v1 (CV vector): [ 0.21321434 -0.22142796  0.2116971  -0.33446565 -0.4262473  -0.04524133
  0.19885267 -0.16503912 -0.26369074 -0.15190415  0.42369696  0.21966852
  0.03887178  0.08007418  0.21436514 -0.11143193 -0.59838694  0.26307216
 -0.16932689  0.09004417 -0.76221895  0.17477512  0.2535282  -0.86055857
 -0.25286117 -0.26593503  0.51129496 -0.058488    0.9443745  -0.4605357
  0.19516094  0.00518642 -0.50468737 -0.4272709  -0.24756731 -0.46951267
  0.18340693  0.2840015  -0.14321882  0.4279297  -0.0746206  -0.00127734
 -0.26268497 -0.5482474  -0.79058087 -0.2576207  -0.37958285 -0.6920545
 -0.01455264 -0.21593226  0.06846645 -0.01554809  0.23201512  0.19747488
  0.75052404  0.1777384   0.52493984  0.22536877  0.43757173  0.45702225
 -0.8085681   0.02853793 -1.0276564   0.37121996  0.19669098 -0.20566857
  0.59110284 -0.08170218  0.17980243  0.0768702  -0.0112029  -0.07267099
 -0.09340509 -0.70791996  0.4260606  -0.01980962 -0.31986737  0.4580304
  0.709492    0.05514605  0.12851556 -

#### 8. Visualization and Notification

In [19]:
# Visualization
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = similarity,
    mode = "gauge+number",
    title = {'text': "Matching percentage (%)"},
    #delta = {'reference': 100},
    gauge = {
        'axis': {'range': [0, 100]},
        'steps' : [
            {'range': [0, 50], 'color': "#FFB6C1"},
            {'range': [50, 70], 'color': "#FFFFE0"},
            {'range': [70, 100], 'color': "#90EE90"}
        ],
             'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 100}}))

fig.update_layout(width=600, height=400)  # Adjust the width and height as desired
fig.show()

# Print notification
if similarity < 50:
    print(colored("Low chance, need to modify your CV!", "red", attrs=["bold"]))
elif similarity >= 50 and similarity < 80:
    print(colored("Good chance but you can improve further!", "yellow", attrs=["bold"]))
else:
    print(colored("Excellent! You can submit your CV.", "green", attrs=["bold"]))

Low chance, need to modify your CV!
